In [1]:
import sys
sys.path.append('../')

In [2]:
from Scripts.pinn_model import TrafficFlowForLWR_PINN, TrafficFlowForARZ_PINN
from Scripts.physics import lwr_pde_residual, physics_loss_calculator
import torch
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
x_data = pd.read_csv('peachtree_dir2_lane1_10m_1s_10min.csv')

In [4]:
x = torch.tensor(x_data['x_nd'].values, dtype=torch.float32).reshape(-1,1)
t = torch.tensor(x_data['t_nd'].values, dtype=torch.float32).reshape(-1,1)
rho = torch.tensor(x_data['rho_nd'].values, dtype=torch.float32).reshape(-1,1)
u = torch.tensor(x_data['u_nd'].values, dtype=torch.float32).reshape(-1,1)
mask = torch.tensor(x_data['mask'].values, dtype=torch.bool).reshape(-1,1)

In [5]:
valid_idx = mask.squeeze(1)
x, t, rho, u = x[valid_idx], t[valid_idx], rho[valid_idx], u[valid_idx]

In [6]:
""" ### LWR Model Approximation using PINNs ###"""

model = TrafficFlowForLWR_PINN(hidden_layers=8, neurons_per_layer=20)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

alpha = 1
beta = 1
num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()

    # Data loss on known density values only
    data_loss = torch.mean((model(x, t) - rho)**2)
    # Physics loss
    physics_loss = torch.mean(lwr_pde_residual(model, x, t)**2)
    total_loss = alpha * data_loss + beta * physics_loss

    total_loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item():.6f}")

Epoch 0, Loss: 0.155990
Epoch 100, Loss: 0.046716
Epoch 200, Loss: 0.037366
Epoch 300, Loss: 0.037042
Epoch 400, Loss: 0.036966
Epoch 500, Loss: 0.036922
Epoch 600, Loss: 0.036896
Epoch 700, Loss: 0.037008
Epoch 800, Loss: 0.036859
Epoch 900, Loss: 0.036903


In [7]:
# Initialize ARZ model
model = TrafficFlowForARZ_PINN(hidden_layers=8, neurons_per_layer=20)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000

for epoch in range(num_epochs):
    optimizer.zero_grad()

    pred = model(x, t)
    rho_pred, u_pred = pred[:, 0:1], pred[:, 1:2]

    # Data loss for density and velocity with real values
    data_loss_rho = torch.mean((rho_pred - rho)**2)
    data_loss_u = torch.mean((u_pred - u)**2)

    # Physics loss
    physics_loss = physics_loss_calculator(x, t, model, beta1=1.0, beta2=1.0, 
                                         gamma1=1.0, gamma2=1.0, tau=0.02)

    total_loss = data_loss_rho + data_loss_u + physics_loss

    total_loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss.item():.6f}")

Epoch 0, Loss: 5.588935
Epoch 100, Loss: 0.439679
Epoch 200, Loss: 0.438705
Epoch 300, Loss: 0.438464
Epoch 400, Loss: 0.438259
Epoch 500, Loss: 0.438064
Epoch 600, Loss: 0.437879
Epoch 700, Loss: 0.437705
Epoch 800, Loss: 0.437540
Epoch 900, Loss: 0.437383
